# Tutorial 12 - Stokes

In [ ]:
import gmsh
import multiphenicsx.io
import multiphenicsx.mesh

## 1. Mesh generation

In [ ]:
mesh_size = 1e-1

In [ ]:
subdomains_vertices = [
    [(1.0, 2.0), (1.0, 1.0), (2.0, 1.0)],  # subdomain 1
    [(2.0, 1.0), (2.0, 2.0), (1.0, 2.0)],  # subdomain 2
    [(0.0, 3.0), (0.0, 2.0), (1.0, 2.0)],  # subdomain 3
    [(1.0, 2.0), (1.0, 3.0), (0.0, 3.0)],  # subdomain 4
    [(0.0, 2.0), (0.0, 1.0), (1.0, 1.0)],  # subdomain 5
    [(1.0, 1.0), (1.0, 2.0), (0.0, 2.0)],  # subdomain 6
    [(0.0, 1.0), (0.0, 0.0), (1.0, 0.0)],  # subdomain 7
    [(1.0, 0.0), (1.0, 1.0), (0.0, 1.0)]  # subdomain 8
]

In [ ]:
boundaries_vertices = [
    [(2.0, 1.0), (2.0, 2.0)],  # boundary 1, inlet
    [(0.0, 0.0), (1.0, 0.0)],  # boundary 2, outlet
    [[(0.0, 3.0), (0.0, 2.0)],
     [(0.0, 2.0), (0.0, 1.0)],
     [(0.0, 1.0), (0.0, 0.0)],
     [(1.0, 3.0), (0.0, 3.0)],
     [(1.0, 2.0), (1.0, 3.0)],
     [(2.0, 2.0), (1.0, 2.0)],
     [(1.0, 1.0), (2.0, 1.0)],
     [(1.0, 0.0), (1.0, 1.0)]],  # boundary 3, walls
]

In [ ]:
gmsh.initialize()
gmsh.model.add("stokes")

In [ ]:
gmsh_points = dict()
for subdomain_vertices in subdomains_vertices:
    for vertex in subdomain_vertices:
        try:
            gmsh_points[vertex]
        except KeyError:
            gmsh_points[vertex] = gmsh.model.geo.addPoint(vertex[0], vertex[1], 0.0, mesh_size)

In [ ]:
gmsh_lines = dict()
gmsh_subdomains = list()
for subdomain_vertices in subdomains_vertices:
    subdomain_lines = list()
    for v in range(3):
        key = (gmsh_points[subdomain_vertices[v]], gmsh_points[subdomain_vertices[(v + 1) % 3]])
        try:
            gmsh_lines[key]
        except KeyError:
            gmsh_lines[key] = gmsh.model.geo.addLine(*key)
            gmsh_lines[key[1], key[0]] = - gmsh_lines[key]
        subdomain_lines.append(gmsh_lines[key])
    subdomain_lines_loop = gmsh.model.geo.addCurveLoop(subdomain_lines)
    gmsh_subdomains.append(gmsh.model.geo.addPlaneSurface([subdomain_lines_loop]))

In [ ]:
gmsh.model.geo.synchronize()
for (label, gmsh_subdomain) in enumerate(gmsh_subdomains):
    gmsh.model.addPhysicalGroup(2, [gmsh_subdomain], label + 1)
for (label, boundary) in enumerate(boundaries_vertices):
    if isinstance(boundary[0], tuple):
        gmsh.model.addPhysicalGroup(
            1, [gmsh_lines[gmsh_points[boundary[0]], gmsh_points[boundary[1]]]], label + 1)
    elif isinstance(boundary[0], list):
        gmsh.model.addPhysicalGroup(
            1, [gmsh_lines[gmsh_points[boundary_[0]], gmsh_points[boundary_[1]]] for boundary_ in boundary],
            label + 1)
gmsh.model.mesh.generate(2)

In [ ]:
mesh, subdomains, boundaries = multiphenicsx.mesh.gmsh_to_fenicsx(gmsh.model, gdim=2)
gmsh.finalize()

In [ ]:
multiphenicsx.io.plot_mesh(mesh)

In [ ]:
multiphenicsx.io.plot_mesh_tags(subdomains)

In [ ]:
multiphenicsx.io.plot_mesh_tags(boundaries)